# 1. Utils (copied from the repository)

In [0]:
import scipy.io
import math
import numpy as np
from functools import reduce
from scipy.interpolate import griddata
from sklearn.preprocessing import scale

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, LSTM, Concatenate
from tensorflow.keras.layers import Input, TimeDistributed, Reshape, MaxPooling1D, Permute, Conv1D, BatchNormalization
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def cart2sph(x, y, z):
    """
    Transform Cartesian coordinates to spherical
    :param x: X coordinate
    :param y: Y coordinate
    :param z: Z coordinate
    :return: radius, elevation, azimuth
    """
    x2_y2 = x**2 + y**2
    r = math.sqrt(x2_y2 + z**2)                    # r
    elev = math.atan2(z, math.sqrt(x2_y2))         # Elevation
    az = math.atan2(y, x)                          # Azimuth
    return r, elev, az


def pol2cart(theta, rho):
    """
    Transform polar coordinates to Cartesian
    :param theta: angle value
    :param rho: radius value
    :return: X, Y
    """
    return rho * math.cos(theta), rho * math.sin(theta)


def azim_proj(pos):
    """
    Computes the Azimuthal Equidistant Projection of input point in 3D Cartesian Coordinates.
    Imagine a plane being placed against (tangent to) a globe. If
    a light source inside the globe projects the graticule onto
    the plane the result would be a planar, or azimuthal, map
    projection.

    :param pos: position in 3D Cartesian coordinates
    :return: projected coordinates using Azimuthal Equidistant Projection
    """
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, math.pi / 2 - elev)

def gen_images(locs, features, n_gridpoints, normalize=True,
               augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
    """
    Generates EEG images given electrode locations in 2D space and multiple feature values for each electrode

    :param locs: An array with shape [n_electrodes, 2] containing X, Y
                        coordinates for each electrode.
    :param features: Feature matrix as [n_samples, n_features]
                                Features are as columns.
                                Features corresponding to each frequency band are concatenated.
                                (alpha1, alpha2, ..., beta1, beta2,...)
    :param n_gridpoints: Number of pixels in the output images
    :param normalize:   Flag for whether to normalize each band over all samples
    :param augment:     Flag for generating augmented images
    :param pca:         Flag for PCA based data augmentation
    :param std_mult     Multiplier for std of added noise
    :param n_components: Number of components in PCA to retain for augmentation
    :param edgeless:    If True generates edgeless images by adding artificial channels
                        at four corners of the image with value = 0 (default=False).
    :return:            Tensor of size [samples, colors, W, H] containing generated
                        images.
    """
    feat_array_temp = []
    nElectrodes = locs.shape[0]     # Number of electrodes

    # Test whether the feature vector length is divisible by number of electrodes
    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1] // nElectrodes
    for c in range(n_colors):
        feat_array_temp.append(features[:, c * nElectrodes : nElectrodes * (c+1)])
    if augment:
        if pca:
            for c in range(n_colors):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=True, n_components=n_components)
        else:
            for c in range(n_colors):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=False, n_components=n_components)
    n_samples = features.shape[0]

    # Interpolate the values
    grid_x, grid_y = np.mgrid[
                     min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j,
                     min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j
                     ]
    temp_interp = []
    for c in range(n_colors):
        temp_interp.append(np.zeros([n_samples, n_gridpoints, n_gridpoints]))

    # Generate edgeless images
    if edgeless:
        min_x, min_y = np.min(locs, axis=0)
        max_x, max_y = np.max(locs, axis=0)
        locs = np.append(locs, np.array([[min_x, min_y], [min_x, max_y], [max_x, min_y], [max_x, max_y]]), axis=0)
        for c in range(n_colors):
            feat_array_temp[c] = np.append(feat_array_temp[c], np.zeros((n_samples, 4)), axis=1)

    # Interpolating
    for i in range(n_samples):
        for c in range(n_colors):
            temp_interp[c][i, :, :] = griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),
                                               method='cubic', fill_value=np.nan)
        print('Interpolating {0}/{1}\r'.format(i + 1, n_samples), end='\r')

    # Normalizing
    for c in range(n_colors):
        if normalize:
            temp_interp[c][~np.isnan(temp_interp[c])] = \
                scale(temp_interp[c][~np.isnan(temp_interp[c])])
        temp_interp[c] = np.nan_to_num(temp_interp[c])
    return np.swapaxes(np.asarray(temp_interp), 0, 1)     # swap axes to have [samples, colors, W, H]

def reformatInput(data, labels, indices):
    """
    Receives the indices for train and test datasets.
    param indices: tuple of (train, test) index numbers
    Outputs the train, validation, and test data and label datasets.
    """
    np.random.shuffle(indices[0])
    np.random.shuffle(indices[0])
    trainIndices = indices[0][len(indices[1]):]
    validIndices = indices[0][:len(indices[1])]
    testIndices = indices[1]

    return [(data[trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
            (data[validIndices], np.squeeze(labels[validIndices]).astype(np.int32)),
            (data[testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]



def print_statistics(y_test, y_test_predict, y_val=None, y_val_predict=None):
  if y_val is not None and y_val_predict is not None:
    print('Validation Accuracy score: ', accuracy_score(y_val, y_val_predict))
  print('Test Accuracy score: ', accuracy_score(y_test, y_test_predict))
  print('Test Precision score: ', precision_score(y_test, y_test_predict, average=None))
  print('Test Recall score: ', recall_score(y_test, y_test_predict, average=None))
  print('Test F1 score: ', f1_score(y_test, y_test_predict, average=None))

def load_data():
  # Load electrode locations
  print('Loading data...')
  locs = scipy.io.loadmat('Sample data/Neuroscan_locs_orig.mat')
  locs_3d = locs['A']
  locs_2d = []
  # Convert to 2D
  for e in locs_3d:
      locs_2d.append(azim_proj(e))

  feats = scipy.io.loadmat('Sample data/FeatureMat_timeWin.mat')['features']
  subj_nums = np.squeeze(scipy.io.loadmat('Sample data/trials_subNums.mat')['subjectNum'])

  # Leave-Subject-Out cross validation
  fold_pairs = [] # 13 arrays each of which contain 2 arrays: 1 holds indices of 1 subject and the other array holds the rest indices
  for i in np.unique(subj_nums):
      ts = subj_nums == i
      tr = np.squeeze(np.nonzero(np.bitwise_not(ts)))
      ts = np.squeeze(np.nonzero(ts))
      np.random.shuffle(tr)  # Shuffle indices
      np.random.shuffle(ts)
      fold_pairs.append((tr, ts))
  return feats, subj_nums, fold_pairs, locs_2d

# 2. Data extraction

In [2]:
feats, subj_nums, fold_pairs, locs_2d = load_data()

Loading data...


### 2.1 Label balance 

In [3]:
labels, sample_numbers = np.unique(feats[:, -1], return_counts=True)
label_distributions = zip(labels, np.round(sample_numbers* 100 / sum(sample_numbers), 1))
print("Label\tPercentage")
for label_distribution in label_distributions:
  print("{}\t{}%".format(label_distribution[0], label_distribution[1]))


Label	Percentage
1.0	28.4%
2.0	26.8%
3.0	24.0%
4.0	20.9%


### 2.2 Generating images

In [4]:
# Find the average response over time windows
av_feats = reduce(lambda x, y: x + y, [feats[:, i * 192 : (i + 1) * 192] for i in range(feats.shape[1] // 192)])
av_feats = av_feats / (feats.shape[1] / 192)
images = gen_images(np.array(locs_2d), av_feats, 32, normalize=True)
images = np.transpose(images, (0, 3, 2, 1))
images.shape

(2670, 32, 32, 3)

In [8]:
images_timewin = np.array([gen_images(np.array(locs_2d), feats[:, i * 192:(i + 1) * 192], 32, normalize=True) for i in range(feats.shape[1] // 192)])
images_timewin = np.transpose(images_timewin, (1, 0, 3, 4, 2))
images_timewin.shape

(2670, 7, 32, 32, 3)

In [5]:
labels = np.squeeze(feats[:, -1]) - 1
n_classes = len(np.unique(labels))
print('Number of classes =', n_classes)

Number of classes = 4


# 3. Model training

In [0]:
def train_model(model_builder, images, labels, fold_pairs, epochs=10, batch_size=32, callbacks=[], verbose=0, fold_verbose=0):
  mean_val_score = 0
  mean_test_score = 0
  model = model_builder()

  for i in range(len(fold_pairs)):
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = reformatInput(images, labels, fold_pairs[i])
    model.fit(X_train, y_train, verbose=fold_verbose, epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val, y_val), 
              callbacks=callbacks)
    y_test_predict = np.argmax(model.predict(X_test), axis=1)
    y_val_predict = np.argmax(model.predict(X_val), axis=1)

    val_accuracy_score = accuracy_score(y_val, y_val_predict)
    test_accuracy_score = accuracy_score(y_test, y_test_predict)

    mean_val_score += val_accuracy_score
    mean_test_score += test_accuracy_score

    if verbose >= 2:
      print('===========Fold {}/{}==========='.format(i + 1, len(fold_pairs)))
      print('Validation Accuracy score: ', val_accuracy_score)
      print('Test Accuracy score: ', test_accuracy_score)
    if verbose == 3:
      print('Test Precision score: ', precision_score(y_test, y_test_predict, average=None))
      print('Test Recall score: ', recall_score(y_test, y_test_predict, average=None))
      print('Test F1 score: ', f1_score(y_test, y_test_predict, average=None))

  if verbose >= 1:
    print('\nMean validation accuracy score: ', mean_val_score / len(fold_pairs))
    print('Mean test accuracy score: ', mean_test_score / len(fold_pairs))


## 3.1 ConvNet Arch. A


In [8]:
def conv_net_A():
  model = Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
      MaxPooling2D(pool_size=(2, 2), strides=2),
      Flatten(),
      Dropout(0.5),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(n_classes, activation='softmax')
      ]    
  )

  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_A, images, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.972972972972973
Test Accuracy score:  0.5135135135135135
===========Fold 2/13===========
Validation Accuracy score:  0.9528301886792453
Test Accuracy score:  0.7216981132075472
===========Fold 3/13===========
Validation Accuracy score:  0.9547738693467337
Test Accuracy score:  0.9748743718592965
===========Fold 4/13===========
Validation Accuracy score:  0.9353233830845771
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9132653061224489
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.945273631840796
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9481865284974094
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9504950495049505
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9523809523809523
Test Accuracy score:  1.0
===========F

## 3.2 ConvNet Arch. B

In [32]:
def conv_net_B():
  model = Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
      MaxPooling2D(pool_size=(2, 2), strides=2),
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
      MaxPooling2D(pool_size=(2, 2), strides=2),
      Flatten(),
      Dropout(0.5),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(n_classes, activation='softmax')  
      ]
  )

  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_B, images, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9459459459459459
Test Accuracy score:  0.4972972972972973
===========Fold 2/13===========
Validation Accuracy score:  0.9339622641509434
Test Accuracy score:  0.6933962264150944
===========Fold 3/13===========
Validation Accuracy score:  0.9447236180904522
Test Accuracy score:  0.9698492462311558
===========Fold 4/13===========
Validation Accuracy score:  0.9502487562189055
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9336734693877551
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.945273631840796
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9326424870466321
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9554455445544554
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9571428571428572
Test Accuracy score:  1.0
===========

## 3.3 ConvNet Arch. C

In [33]:
def conv_net_C():
  model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax')
    ]
  )

  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_C, images, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9405405405405406
Test Accuracy score:  0.5513513513513514
===========Fold 2/13===========
Validation Accuracy score:  0.9433962264150944
Test Accuracy score:  0.7735849056603774
===========Fold 3/13===========
Validation Accuracy score:  0.914572864321608
Test Accuracy score:  0.9849246231155779
===========Fold 4/13===========
Validation Accuracy score:  0.9303482587064676
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9336734693877551
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.9402985074626866
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9637305699481865
Test Accuracy score:  0.9948186528497409
===========Fold 8/13===========
Validation Accuracy score:  0.9752475247524752
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9571428571428572
Test Accuracy score:  

## 3.4 ConvNet Arch. D

In [34]:
def conv_net_D():
  model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax')                
    ]    
  )


  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_D, images, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9405405405405406
Test Accuracy score:  0.5837837837837838
===========Fold 2/13===========
Validation Accuracy score:  0.9433962264150944
Test Accuracy score:  0.75
===========Fold 3/13===========
Validation Accuracy score:  0.9346733668341709
Test Accuracy score:  0.9095477386934674
===========Fold 4/13===========
Validation Accuracy score:  0.945273631840796
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9438775510204082
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.9353233830845771
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9222797927461139
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9653465346534653
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9428571428571428
Test Accuracy score:  1.0
===========Fold 10/13====

## 3.5 ConvNet+Maxpool

In [13]:
def conv_net_max_pool():
  model = Sequential([
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(7, 32, 32, 3))),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=2)),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=2)),
    TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=2)),
    Reshape((7, 4*4*128)),
    MaxPooling1D(pool_size=7, strides=1, data_format='channels_last'),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax')                  
    ]    
  )


  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_max_pool, images_timewin, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9567567567567568
Test Accuracy score:  0.572972972972973
===========Fold 2/13===========
Validation Accuracy score:  0.9528301886792453
Test Accuracy score:  0.6933962264150944
===========Fold 3/13===========
Validation Accuracy score:  0.8994974874371859
Test Accuracy score:  0.9698492462311558
===========Fold 4/13===========
Validation Accuracy score:  0.9651741293532339
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9387755102040817
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.9402985074626866
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9844559585492227
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9603960396039604
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9619047619047619
Test Accuracy score:  1.0
===========

## 3.6 ConvNet+1D-Conv

In [38]:
def conv_net_1d_conv():
  model= Sequential([
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(7, 32, 32, 3))),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Flatten()),
    Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='valid'),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax'),
    ]    
  )

  adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_1d_conv, images_timewin, labels, fold_pairs, epochs=40, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9621621621621622
Test Accuracy score:  0.5459459459459459
===========Fold 2/13===========
Validation Accuracy score:  0.9386792452830188
Test Accuracy score:  0.7735849056603774
===========Fold 3/13===========
Validation Accuracy score:  0.964824120603015
Test Accuracy score:  0.9949748743718593
===========Fold 4/13===========
Validation Accuracy score:  0.9751243781094527
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9693877551020408
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.9701492537313433
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9689119170984456
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9405940594059405
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9333333333333333
Test Accuracy score:  1.0
===========

## 3.7 ConvNet+LSTM

In [39]:
class WeightClip(Constraint):
    '''Clips the weights by value c
    '''
    def __init__(self, c):
        self.c = c

    def __call__(self, p):
        return tf.keras.backend.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}

def conv_net_lstm():
  model= Sequential([
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(7, 32, 32, 3))),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))),
    TimeDistributed(Flatten()),
    LSTM(128, activation='tanh', kernel_constraint=WeightClip(100)), 
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(n_classes, activation='softmax'),
    ]    
  )

  #changed learning rate to 0.0001, otherwise doesn't converge
  adam_optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_lstm, images_timewin, labels, fold_pairs, epochs=10, callbacks=[EarlyStopping(patience=10)], verbose=2)

===========Fold 1/13===========
Validation Accuracy score:  0.9675675675675676
Test Accuracy score:  0.5567567567567567
===========Fold 2/13===========
Validation Accuracy score:  0.9481132075471698
Test Accuracy score:  0.7547169811320755
===========Fold 3/13===========
Validation Accuracy score:  0.9447236180904522
Test Accuracy score:  0.9246231155778895
===========Fold 4/13===========
Validation Accuracy score:  0.9203980099502488
Test Accuracy score:  1.0
===========Fold 5/13===========
Validation Accuracy score:  0.9540816326530612
Test Accuracy score:  1.0
===========Fold 6/13===========
Validation Accuracy score:  0.9353233830845771
Test Accuracy score:  1.0
===========Fold 7/13===========
Validation Accuracy score:  0.9326424870466321
Test Accuracy score:  1.0
===========Fold 8/13===========
Validation Accuracy score:  0.9702970297029703
Test Accuracy score:  1.0
===========Fold 9/13===========
Validation Accuracy score:  0.9333333333333333
Test Accuracy score:  1.0
==========

## 3.8 ConvNet+LSTM/1D-Conv

In [62]:
def conv_net_lstm_1d_conv():
  input_layer = Input(shape=(7, 32, 32, 3))
  conv2d = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')) (input_layer)
  conv2d = TimeDistributed(BatchNormalization()) (conv2d)
  conv2d = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')) (conv2d)
  conv2d = TimeDistributed(BatchNormalization()) (conv2d)
  conv2d = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')) (conv2d)
  conv2d = TimeDistributed(BatchNormalization()) (conv2d)
  conv2d = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')) (conv2d)
  maxpool2d = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) (conv2d)
  maxpool2d = TimeDistributed(BatchNormalization()) (maxpool2d)
  conv2d = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')) (maxpool2d)
  conv2d = TimeDistributed(BatchNormalization()) (conv2d)
  conv2d = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')) (conv2d)
  maxpool2d = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) (conv2d)
  maxpool2d = TimeDistributed(BatchNormalization()) (maxpool2d)
  conv2d = TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')) (maxpool2d)
  maxpool2d = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) (conv2d)
  maxpool2d = TimeDistributed(BatchNormalization()) (maxpool2d)
  maxpool2d = TimeDistributed(Flatten()) (maxpool2d)

  lstm = LSTM(128, activation='tanh', kernel_constraint=WeightClip(100)) (maxpool2d)

  conv1d = Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='valid') (maxpool2d)
  conv1d = Flatten() (conv1d)

  concat_layer = Concatenate()([lstm, conv1d])
  concat_layer = BatchNormalization() (concat_layer)

  layer = Dropout(0.5) (concat_layer)
  layer = Dense(256, activation='relu') (layer)
  layer = BatchNormalization() (layer)
  layer = Dropout(0.5) (layer)
  output_layer = Dense(n_classes, activation='softmax') (layer)

  model = Model(inputs=[input_layer], outputs=[output_layer])

  adam_optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
  model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

train_model(conv_net_lstm_1d_conv, images_timewin, labels, fold_pairs, epochs=6, callbacks=[EarlyStopping(patience=6)], verbose=3)

===========Fold 1/13===========
Validation Accuracy score:  0.8540540540540541
Test Accuracy score:  0.9027027027027027
Test Precision score:  [0.93103448 0.88461538 0.86538462 0.95652174]
Test Recall score:  [0.94736842 0.95833333 0.95744681 0.66666667]
Test F1 score:  [0.93913043 0.92       0.90909091 0.78571429]
===========Fold 2/13===========
Validation Accuracy score:  0.9292452830188679
Test Accuracy score:  0.839622641509434
Test Precision score:  [0.84210526 0.6        1.         1.        ]
Test Recall score:  [1.         0.73333333 0.61111111 0.97959184]
Test F1 score:  [0.91428571 0.66       0.75862069 0.98969072]
===========Fold 3/13===========
Validation Accuracy score:  0.949748743718593
Test Accuracy score:  0.949748743718593
Test Precision score:  [0.87301587 1.         0.97727273 0.97916667]
Test Recall score:  [1.         0.81481481 1.         1.        ]
Test F1 score:  [0.93220339 0.89795918 0.98850575 0.98947368]
===========Fold 4/13===========
Validation Accuracy 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
